# Generating Transit Travel Times with r5py

This notebook is modified based on Acess presentation by Dr Willem to calculate access to opportunities using r5py.

This is a generally computationally intensive process, and has been the main technical hurdle to overcome in this process.

In this workbook, we are going to generate transit travel times using a relativley new Python library `r5py`. R5py is designed to allow Python users to access the open-source R5 engine, a powerful engine that is the spiritual successort to OpenTripPlanner. You can ready more about r5py via [their documentation](https://r5py.readthedocs.io), or check out [R5's own github page](https://github.com/conveyal/r5).
In this workbook we are going to generate transit travel time using the Python library r5py.

In our example analysis, we want to answer two questions:
- How is the access to hospitals distributed across different populations?
- How is the access to child care spaces distributed across different populations?
- How does peak-period and evening service change this access for various populations?

For this we need to generate *two* travel time matrices. One for a peak period (7-9am) and for an evening period (9-11pm).

The beauty of the R5 engine is that it allows us to measure a median peak-period value very easily. To do this, we start our analysis at the beginning of our specified time period and set the duration of the analysis.

In [25]:
%pip install geojson

Note: you may need to restart the kernel to use updated packages.


In [26]:
#Imports
import os
import sys
import csv
import math
import datetime
import multiprocessing as mp
from functools import partial

import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
import dill
from tqdm import tqdm

import requests
import json
from geojson import Feature, FeatureCollection, Point

# R5py imports
from r5py import TransportNetwork, TravelTimeMatrixComputer, TransportMode

# AWS
import boto3
import io
from botocore.exceptions import ClientError, NoCredentialsError

# Warnings configuration
import warnings
warnings.filterwarnings('ignore')

In [25]:
#os.environ['GDAL_DATA'] = os.path.join(f'{os.sep}'.join(sys.executable.split(os.sep)[:-1]), 'Library', 'share', 'gdal')

# This sets the amount of memory we are using for R5py calcualtions
#sys.argv.append(["--max-memory", "8G"])

### **Input 1**: Load "Opportunities" datasets
Let's start by loading the appropriate data and setting some key settings for R5py. One little quirk we are going to need is that R5py requires our origin/destination points to be named `id`, not anything else like `dauid`. We'll make that change now.

In [4]:
## Calgary Data
da_centroids = gpd.read_file("data/calgary/da_centroids_with_locations.geojson").rename(columns={"dauid":"id"}) #1675 rows
daycares = gpd.read_file("data/calgary/daycare_locations.geojson") #368 (total: OD = 616400)
#hospitals = gpd.read_file("data/calgary/hospital_locations.geojson") # 5 (OD = 8375)

## Guadalajara Data
#da_centroids = gpd.read_file("data/guadalajara/UnAdj_100m_2020.geojson").rename(columns={"pointid":"id"})#
#dest_po = gpd.read_file("data/guadalajara/guadalajara_greenspace_perimeterpoints.geojson")
#dest_po["id"] = range(len(da_centroids)+ 1, len(da_centroids) + len(dest_po) + 1) #adding id column

print(len(da_centroids))
print(len(daycares))
#print(len(dest_po)) 

1675
368


### **Input 2**: Set Up Transport Network

To calculate travel times, we need to set up a transport network (which happens to be called a `TransportNetwork` class). The transport network needs both an underlying OpenStreetMap PBF file as well as one or more GTFS feeds. So let's go ahead and set up our transport network, which takes as its first argument the path to our PBF file and as a second argument a list of paths to GTFS files (of which we only have one in Calgary).

In [5]:
def create_transport_network(city_name):
    """
    Create a TransportNetwork object for a given city.  
    city_name (str): Name of the city (used for folder naming)
    TransportNetwork: A TransportNetwork object for the specified city
    """
    base_path = os.path.join("data", city_name)
    
    # Find the .osm.pbf file
    osm_files = [f for f in os.listdir(base_path) if f.endswith('.osm.pbf')]
    if not osm_files:
        raise FileNotFoundError(f"No .osm.pbf file found in {base_path}")
    osm_file = os.path.join(base_path, osm_files[0])
    
    # Find the GTFS zip file
    gtfs_files = [f for f in os.listdir(base_path) if f.endswith('.zip')]
    if not gtfs_files:
        raise FileNotFoundError(f"No GTFS zip file found in {base_path}")
    gtfs_file = os.path.join(base_path, gtfs_files[0])
    
    return TransportNetwork(osm_file, [gtfs_file])


# Function call
city_name = "calgary"  # or "guadalajara"
transport_network = create_transport_network(city_name)

This will build us a transport network which we can use to compute travel times. So let's go ahead and do that next!

In a smaller test, we would create a travel time matrix computer (`TravelTimeMatrixComputer`) which lets us specify a whole bunch of potential parameters, most importantly origins and destinations. Our origins are the DA centroids, and our (first) destination will be the hospital centroids.

In [28]:
%%time
## Guadalajara example
#travel_time_computer = TravelTimeMatrixComputer(
#    transport_network,
#    origins=da_centroids,
#    destinations=dest_po,
#    departure=datetime.datetime(2021, 1, 1, 7, 0),
#    departure_time_window=datetime.timedelta(minutes=10),
#    max_time=datetime.timedelta(minutes=30),
#    transport_modes=[TransportMode.TRANSIT, TransportMode.WALK]
#)

## Calgary example (with daycares as destinations)
# travel_time_computer = TravelTimeMatrixComputer(
#    transport_network,
#    origins=da_centroids,
#    destinations=daycares,
#    departure=datetime.datetime(2023, 3, 15, 7, 0),
#    departure_time_window=datetime.timedelta(hours=2),
#    transport_modes=[TransportMode.TRANSIT, TransportMode.WALK]
# )

# travel_time_computer.compute_travel_times()

CPU times: total: 0 ns
Wall time: 0 ns


### Compute the Travel Times
This will take a little while to run, and we'll write the matrix directly to a file (saved in S3).

In [21]:
# First S3 related codes!
try:
    s3_client = boto3.client('s3')
    S3_BUCKET = 'wri-cities-sandbox'
    S3_PREFIX = 'r5py_sandbox/'
    USE_S3 = True
except NoCredentialsError:
    print("AWS credentials not found. Will use local storage instead.")
    USE_S3 = False

def check_s3_permissions():
    """
    Check if we have the necessary permissions to access the S3 bucket.
    """
    if not USE_S3:
        return False
    try:
        s3_client.head_bucket(Bucket=S3_BUCKET)
        return True
    except (ClientError, NoCredentialsError) as e:
        print(f"Error accessing S3 bucket: {e}")
        return False

def save_to_s3(df, file_name, mode="a"):
    """
    Save a DataFrame to S3 as a CSV file.
    """
    try:
        if mode == 'a':
            try:
                obj = s3_client.get_object(Bucket=S3_BUCKET, Key=f"{S3_PREFIX}{file_name}")
                existing_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
                df = pd.concat([existing_df, df], ignore_index=True)
            except ClientError as e:
                if e.response['Error']['Code'] != 'NoSuchKey':
                    raise
                # If NoSuchKey, we're creating a new file, so just proceed with the current df

        csv_buffer = io.StringIO()
        df.to_csv(csv_buffer, index=False)
        s3_client.put_object(
            Bucket=S3_BUCKET,
            Key=f"{S3_PREFIX}{file_name}",
            Body=csv_buffer.getvalue())
        
    except Exception as e:
        print(f"Error saving to S3: {e}. Saving locally instead.")
        save_locally(df, file_name, mode)


def save_locally(df, city_name, file_name, mode='a'):
    output_dir = os.path.join('data', city_name, 'output')
    os.makedirs(output_dir, exist_ok=True)
    local_path = os.path.join(output_dir, file_name)
    df.to_csv(local_path, mode=mode, header=(mode=='w'), index=False)

def save_or_append_data(df, city_name, file_name, mode='a'):
    if USE_S3 and check_s3_permissions():
        save_to_s3(df, file_name, mode)
    else:
        save_locally(df, city_name, file_name, mode)


In [22]:
def calculate_num_batches(geodataframe, batch_size):
    return math.ceil(len(geodataframe) / batch_size)

def process_batch(c_gdf1, c_gdf2, transport_network):
    """"returns a df. """
    travel_time_computer_c = TravelTimeMatrixComputer(
                transport_network,
                origins = c_gdf1,
                destinations = c_gdf2,
                departure = datetime.datetime(2023, 3, 15, 7, 0),  #NEEDS TO MODIFY
                departure_time_window = datetime.timedelta(hours=2), #NEEDS TO MODIFY
                transport_modes = [TransportMode.TRANSIT, TransportMode.WALK]) #NEEDS TO MODIFY
    return travel_time_computer_c.compute_travel_times()  

def process_batches(origin, destinations, BS_O, BS_D, base_file_name, transport_network, city_name):
    """
    Process the data in batches and append results to a CSV file in S3.
    """
    # Generate a unique file name for each run
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = f"{base_file_name}_{timestamp}.csv"
    
    # Check S3 permissions
    if check_s3_permissions():
        storage_type = "S3"
    else:
        storage_type = "local"
        print("S3 access not available. Using local storage.")


    # Calculate the number of batches
    num_b_o = calculate_num_batches(origin, BS_O)
    num_b_d = calculate_num_batches(destinations, BS_D)

    # Compute first geodataframe in batches
    for i in range(num_b_o):
        start_idx_gdf1 = i * BS_O
        end_idx_gdf1 = min(start_idx_gdf1 + BS_O, len(origin))
        chunk_gdf1 = origin.iloc[start_idx_gdf1:end_idx_gdf1]

        # Compute second geodataframe in batches
        for j in range(num_b_d):
            start_idx_gdf2 = j * BS_D
            end_idx_gdf2 = min(start_idx_gdf2 + BS_D, len(destinations))
            chunk_gdf2 = destinations.iloc[start_idx_gdf2:end_idx_gdf2]

            
            # MAJOR compute        
            travel_times_df = process_batch(chunk_gdf1, chunk_gdf2, transport_network)

            # Append the current batch to the CSV file in S3
            save_or_append_data(travel_times_df, city_name, file_name, mode='a')

            print(f"Processed and saved batch: origin {i+1}/{num_b_o}, destination {j+1}/{num_b_d}")

    storage_type = "S3" if USE_S3 else "local"
    print(f"All batches processed and saved to {storage_type} storage: {file_name}")
    return file_name


In [23]:
%%time
#Usage
city_name = 'calgary'
base_file_name = 'travel_times'

## Check if this is already called in the above cell. Better to call it above instead of here. This call takes some time.
# transport_network = create_transport_network("Calgary") 

# Define the batch size for processing
BS_O = 1000  # Adjust as needed - origin
BS_D = 1000     # Adjust as needed - destination


# Process batches and get the final DataFrame
result = process_batches(da_centroids, daycares, BS_O, BS_D, base_file_name, transport_network, city_name)
print("FIN!")

Error accessing S3 bucket: Unable to locate credentials
S3 access not available. Using local storage.
Error accessing S3 bucket: Unable to locate credentials
Processed and saved batch: origin 1/2, destination 1/1
Error accessing S3 bucket: Unable to locate credentials
Processed and saved batch: origin 2/2, destination 1/1
All batches processed and saved to S3 storage: travel_times_20240917_191840.csv
FIN!
CPU times: total: 3min 41s
Wall time: 3min 37s


### **Using Spatial Index**

Can ignore this part for now... :D

In [32]:
import h3
def find_dest_h3(orig, dest, h3_res=9, ring_size=3):
    # h3_res = 7: ~5.2km^2; h3_res = 8: ~0.74km^2; h3_res = 9: ~0.11km^2
    # A k-ring of size k around a central hexagon includes all hexagons that are within k steps from the center.
    # When ring_size=1, the k-ring includes: (1) The central hexagon; (2) the six hexagons surrounding the central hex. (total of 7 hexs)
    # ring_size=2 means we have about 19 hexs

    # Ensure same CRS
    orig = orig.to_crs("EPSG:4326")
    dest = dest.to_crs("EPSG:4326")
    
    # Convert origins and destinations to H3 indexes
    orig["h3_i"] = orig.apply(lambda i: h3.geo_to_h3(i.geometry.y, i.geometry.x, h3_res), axis=1)
    dest["h3_i"] = dest.apply(lambda i: h3.geo_to_h3(i.geometry.y, i.geometry.x, h3_res), axis=1)
    
    results = []

    orig_rings = orig.apply(lambda row: set(h3.k_ring(row["h3_i"], ring_size)), axis=1)
    
    for idx, o_ring in orig_rings.items():
        o = orig.loc[idx]
        matches = dest[dest["h3_i"].isin(o_ring)]

        for _, d in matches.iterrows():
            results.append({
                "origin_id": o["id"],
                "dest_id": d["id"],
                "o_geometry": o.geometry,
                "d_geometry": d.geometry})
    
    od_pairs = gpd.GeoDataFrame(results, geometry="o_geometry", crs=orig.crs)
    od_pairs["d_geometry"] = gpd.GeoSeries(od_pairs["d_geometry"], crs=orig.crs)

    grouped_od_pairs = od_pairs.groupby("origin_id").agg({"o_geometry": "first", "dest_id": lambda x: list(x), "d_geometry": lambda x: list(x)}).reset_index()
    grouped_od_pairs = gpd.GeoDataFrame(grouped_od_pairs, geometry="o_geometry", crs="EPSG:4326")

    return od_pairs, grouped_od_pairs

def compute_tt(grouped_od_pairs, transport_network, departure_time, time_window, modes):
    """
    Compute travel times for origin-destination pairs using r5py.
    """
    all_results = []

    # Iterate through each origin
    for _, row in tqdm(grouped_od_pairs.iterrows(), total=len(grouped_od_pairs), desc="Processing origins"):

        # Create GeoDataFrame for the current origin
        origin = gpd.GeoDataFrame({"id": [row.origin_id], "geometry": [row.o_geometry]}, crs=grouped_od_pairs.crs)
        
        # Create GeoDataFrame for all destinations for this origin
        destinations = gpd.GeoDataFrame({"id": row.dest_id, "geometry": row.d_geometry}, crs=grouped_od_pairs.crs)

        travel_time_computer = TravelTimeMatrixComputer(
                transport_network,
                origins=origin,
                destinations=destinations,
                departure=departure_time,
                departure_time_window=time_window,
                transport_modes=modes)

        travel_times = travel_time_computer.compute_travel_times()

        # Aggregate results for each origin (taking min)
        # if not travel_times.empty:
        #     min_index = travel_times["travel_time"].idxmin()
        #     min_travel_time = travel_times.loc[min_index, "travel_time"]
        #     min_dest_id = travel_times.loc[min_index, "to_id"]
        #     min_dest_geometry = destinations.loc[destinations["id"] == min_dest_id, "geometry"].iloc[0]

        # all_results.append({"origin_id": row.origin_id,"o_geometry": row.o_geometry,"min_travel_time": min_travel_time,
        #                 "min_dest_id": min_dest_id, "d_geometry": min_dest_geometry})
        all_results.append(travel_times)
           
    return pd.concat(all_results, ignore_index=True)

In [33]:
%%time
## Running the functions with Calgary data
od_pairs, grouped_od_pairs = find_dest_h3(da_centroids, daycares, h3_res=8, ring_size=1)
print(f"Number of OD pairs: {len(od_pairs)}")

print(grouped_od_pairs)

#r5py
network = TransportNetwork("data/calgary/Calgary.osm.pbf", ["data/calgary/cgy-gtfs-2023-03-03.zip"])
departure_time = datetime.datetime(2023, 3, 15, 7, 0)
time_window = datetime.timedelta(hours=2)
modes = [TransportMode.TRANSIT, TransportMode.WALK]

#Compute Travel Time
travel_times_df = compute_tt(grouped_od_pairs, network, departure_time, time_window, modes)
travel_times_df


Number of OD pairs: 6639
     origin_id                   o_geometry                   dest_id  \
0     48060056  POINT (-114.09749 51.13788)   [38, 42, 188, 214, 290]   
1     48060057  POINT (-114.09353 51.13976)   [38, 42, 188, 214, 290]   
2     48060058  POINT (-114.09522 51.13583)   [38, 42, 188, 214, 290]   
3     48060059  POINT (-114.09047 51.13885)   [38, 42, 188, 214, 290]   
4     48060060  POINT (-114.08807 51.13915)   [38, 42, 188, 214, 290]   
...        ...                          ...                       ...   
1538  48062787  POINT (-114.13346 51.07331)  [88, 108, 237, 251, 339]   
1539  48062789  POINT (-114.04997 51.11896)           [126, 145, 154]   
1540  48062790  POINT (-114.06178 51.12943)                [126, 145]   
1541  48062791  POINT (-114.16624 51.04638)        [5, 297, 313, 324]   
1542  48062792     POINT (-114.16901 51.04)        [5, 297, 313, 324]   

                                             d_geometry  
0     [POINT (-114.098894 51.1334222), P

KeyboardInterrupt: 

In [27]:
#download schools in Guadalajara
def get_schools_in_guadalajara():
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = """
    [out:json];
    area["name"="Guadalajara"]["admin_level"="6"]->.searchArea;
    (
      node["amenity"="school"](area.searchArea);
      way["amenity"="school"](area.searchArea);
      relation["amenity"="school"](area.searchArea);
    );
    out center;
    """
    
    response = requests.get(overpass_url, params={'data': overpass_query})
    data = response.json()

    features = []
    for element in data['elements']:
        if element['type'] == 'node':
            lon, lat = element['lon'], element['lat']
        elif 'center' in element:
            lon, lat = element['center']['lon'], element['center']['lat']
        else:
            continue  # Skip elements without coordinates
        
        properties = {
            'id': element['id'],
            'type': element['type'],
            'amenity': 'school'
        }
        properties.update(element.get('tags', {}))
        
        feature = Feature(geometry=Point((lon, lat)), properties=properties)
        features.append(feature)

    feature_collection = FeatureCollection(features)
    
    with open('guadalajara_schools.geojson', 'w', encoding='utf-8') as f:
        json.dump(feature_collection, f, ensure_ascii=False, indent=4)

    print(f"Downloaded {len(features)} schools in Guadalajara")

get_schools_in_guadalajara()

Downloaded 1128 schools in Guadalajara
